## Data Scraping from Wiki

In [1]:
import pandas as pd
import numpy as np
import requests
import datetime 
from bs4 import BeautifulSoup
import requests

In [2]:
import unicodedata

In [3]:
wiki = 'https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches'
response = requests.get(wiki)
soup = BeautifulSoup(response.content,'html')

In [4]:
a = soup.find_all('table')


In [5]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out
def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


In [6]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(static_url)



In [7]:
soup = BeautifulSoup(response.content,'html.parser')


In [8]:
html_tables = soup.find_all('table')
#html_table

In [9]:
first_launch_table = html_tables[2]
#print(first_launch_table)

In [10]:
column_names = [th.get_text(strip = True) for th in first_launch_table.find_all('th')]
print(column_names)

['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']


In [11]:
column_names.remove('Date andtime (UTC)')
column_names

['Flight No.',
 'Version,Booster[b]',
 'Launch site',
 'Payload[c]',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launchoutcome',
 'Boosterlanding',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7']

In [39]:
column_names

['Flight No.',
 'Version,Booster[b]',
 'Launch site',
 'Payload[c]',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launchoutcome',
 'Boosterlanding']

In [12]:
launch_dict= dict.fromkeys(column_names)


In [13]:
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [14]:
launch_dict['Payload mass']

[]

In [28]:
launch_dict

{'Flight No.': '121',
 'Date': '6 June 2021',
 'Time': '04:26',
 'Version Booster': 'F9 B5',
 'Launch Site': 'CCSFS',
 'Payload': 'SXM-8',
 'Payload mass': '7,000 kg',
 'Orbit': 'GTO',
 'Customer': 'GTO',
 'Launch outcome': 'Success\n',
 'Booster landing': 'Success'}

In [63]:
extracted_row = 0
# Create an empty list to store the extracted data for each launch
launch_data_list = []

# Extract each table
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # get table row
    for rows in table.find_all("tr"):
        # check to see if the first table heading is a number corresponding to launch a number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        # get table element
        row = rows.find_all('td')
        # if it is a number save cells in a dictionary
        if flag:
            extracted_row += 1
            # Flight Number value
            # Append the flight_number into launch_dict with key `Flight No.`
            launch_dict = {'Flight No.': flight_number}

            datatimelist = date_time(row[0])

            # Date value
            # Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'] = date

            # Time value
            # Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'] = time

            # Booster version
            # Append the bv into launch_dict with key `Version Booster`
            bv = booster_version(row[1])
            if not (bv):
                bv = row[1].a.string
            launch_dict['Version Booster'] = bv

            # Launch Site
            # Append the launch_site into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict['Launch Site'] = launch_site

            # Payload
            # Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload'] = payload

            # Payload Mass
            # Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'] = payload_mass

            # Orbit
            # Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'] = orbit

            # Customer
            # Append the customer into launch_dict with key `Customer`
            launch_dict['Customer'] = row[6].a.string if row[6].a else None
            #launch_dict['Customer'] = customer

            # Launch outcome
            # Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'] = launch_outcome

            # Booster landing
            # Append the booster_landing into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'] = booster_landing

            # Append the launch_dict to the launch_data_list
            launch_data_list.append(launch_dict)

# Display the resulting list of launch data
for launch_data in launch_data_list:
    a = launch_data
    print(a)

{'Flight No.': '1', 'Date': '4 June 2010', 'Time': '18:45', 'Version Booster': 'F9 v1.0B0003.1', 'Launch Site': 'CCAFS', 'Payload': 'Dragon Spacecraft Qualification Unit', 'Payload mass': 0, 'Orbit': 'LEO', 'Customer': 'SpaceX', 'Launch outcome': 'Success\n', 'Booster landing': 'Failure'}
{'Flight No.': '2', 'Date': '8 December 2010', 'Time': '15:43', 'Version Booster': 'F9 v1.0B0004.1', 'Launch Site': 'CCAFS', 'Payload': 'Dragon', 'Payload mass': 0, 'Orbit': 'LEO', 'Customer': 'NASA', 'Launch outcome': 'Success', 'Booster landing': 'Failure'}
{'Flight No.': '3', 'Date': '22 May 2012', 'Time': '07:44', 'Version Booster': 'F9 v1.0B0005.1', 'Launch Site': 'CCAFS', 'Payload': 'Dragon', 'Payload mass': '525 kg', 'Orbit': 'LEO', 'Customer': 'NASA', 'Launch outcome': 'Success', 'Booster landing': 'Not attempted\n'}
{'Flight No.': '4', 'Date': '8 October 2012', 'Time': '00:35', 'Version Booster': 'F9 v1.0B0006.1', 'Launch Site': 'CCAFS', 'Payload': 'SpaceX CRS-1', 'Payload mass': '4,700 kg', 

In [64]:
launch_dict = launch_data_list
df = pd.DataFrame(launch_dict)
df

,Flight No.,Date,Time,Version Booster,Launch Site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,1,4 June 2010,18:45,F9 v1.0B0003.1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,Failure
1,2,8 December 2010,15:43,F9 v1.0B0004.1,CCAFS,Dragon,0,LEO,NASA,Success,Failure
2,3,22 May 2012,07:44,F9 v1.0B0005.1,CCAFS,Dragon,525 kg,LEO,NASA,Success,Not attempted\n
3,4,8 October 2012,00:35,F9 v1.0B0006.1,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,No attempt
4,5,1 March 2013,15:10,F9 v1.0B0007.1,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,Not attempted\n
...,...,...,...,...,...,...,...,...,...,...,...
116,117,9 May 2021,06:42,F9 B5B1051.10,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,Success
117,118,15 May 2021,22:56,F9 B5B1058.8,KSC,Starlink,"~14,000 kg",LEO,SpaceX,Success\n,Success
118,119,26 May 2021,18:59,F9 B5B1063.2,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,Success
119,120,3 June 2021,17:29,F9 B5B1067.1,KSC,SpaceX CRS-22,"3,328 kg",LEO,NASA,Success\n,Success


In [66]:
#df['Customer'].value_counts()

In [67]:
df.to_csv('spacex_web_scraped.csv', index=False)